# Gaussian Graphical Model on a 4‑Cycle
This notebook simulates a 4‑dimensional Gaussian random vector whose precision matrix corresponds to the cyclic graph 1–2–3–4–1.
We verify conditional independence structure and estimate covariance/precision matrices.

## 1. Define the graph and precision matrix

In [1]:
import numpy as np
import pandas as pd

d = 4
Theta = np.array([
    [10,  5,  0,  3],
    [ 5, 10,  5,  0],
    [ 0,  5, 10,  5],
    [ 3,  0,  5, 10]
])
Sigma = np.linalg.inv(Theta)
Sigma

array([[ 0.29411765, -0.26470588,  0.23529412, -0.20588235],
       [-0.26470588,  0.38823529, -0.31176471,  0.23529412],
       [ 0.23529412, -0.31176471,  0.38823529, -0.26470588],
       [-0.20588235,  0.23529412, -0.26470588,  0.29411765]])

## 2. Simulate Gaussian sample

In [2]:
N = 10**6
mean = np.zeros(d)
X = np.random.multivariate_normal(mean, Sigma, size=N)
X[:5]

array([[-0.21956415,  1.08123084, -1.02922016,  0.76059621],
       [-0.37819471,  0.82693782, -0.06808981,  0.09553774],
       [-0.11572183, -0.2198976 , -0.19600071,  0.85772628],
       [ 0.19268054,  0.38428129, -0.37804683,  0.17186655],
       [ 0.94214134, -1.00244954,  1.04096084, -0.82306598]])

## 3. Standardize margins

In [3]:
std = np.sqrt(np.diag(Sigma))
X = X / std
np.var(X, axis=0)

array([0.99938952, 0.99972868, 1.0017663 , 1.00028548])

## 4. Estimate covariance and precision

In [4]:
EstimSigma = np.cov(X, rowvar=False)
EstimTheta = np.linalg.inv(EstimSigma)
EstimSigma, EstimTheta

(array([[ 0.99939052, -0.78265185,  0.69641576, -0.70053628],
        [-0.78265185,  0.99972968, -0.8038503 ,  0.69652438],
        [ 0.69641576, -0.8038503 ,  1.00176731, -0.78465469],
        [-0.70053628,  0.69652438, -0.78465469,  1.00028648]]),
 array([[2.94223525, 1.68882774, 0.00682204, 0.88993085],
        [1.68882774, 3.88404106, 1.94678738, 0.00531014],
        [0.00682204, 1.94678738, 3.88403098, 1.69593149],
        [0.88993085, 0.00531014, 1.69593149, 2.94960581]]))

## 5. Maximum-likelihood estimation

In [6]:
from scipy.optimize import minimize

def construct_symmetric_matrix(theta_vec):
    Theta_mat = np.zeros((d, d))
    idx = np.triu_indices(d)
    Theta_mat[idx] = theta_vec
    Theta_mat = Theta_mat + Theta_mat.T - np.diag(np.diag(Theta_mat))
    return Theta_mat

def log_likelihood(theta_vec, S):
    Theta_mat = construct_symmetric_matrix(theta_vec)
    det = np.linalg.det(Theta_mat)
    if det <= 0:
        return -np.inf
    return np.log(det) - np.trace(S @ Theta_mat)

A = np.random.randn(d, d)
Theta0 = A @ A.T + d * np.eye(d)
theta0_vec = Theta0[np.triu_indices(d)]

res = minimize(lambda t: -log_likelihood(t, EstimSigma),
               theta0_vec)

Theta_MLE = construct_symmetric_matrix(res.x)
Theta_MLE

C:\Users\20254817\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


array([[2.94223486, 1.68883029, 0.00682449, 0.88993044],
       [1.68883029, 3.88404619, 1.94678662, 0.00530747],
       [0.00682449, 1.94678662, 3.8840208 , 1.6959251 ],
       [0.88993044, 0.00530747, 1.6959251 , 2.94960183]])